하이브리드 추천 시스템의 기본 컨셉

In [18]:
import random
import numpy as np
import chapter8_modules as m8

# 기본 추천기
def recommender0(recomm_list):
  recommendations = []
  for pair in recomm_list:
    recommendations.append(random.random() * 4 + 1)
  return np.array(recommendations)
# 기본 추천기
def recommender1(recomm_list):
  recommendations = []
  for pair in recomm_list:
    recommendations.append(random.random() * 4 + 1)
  return np.array(recommendations)

_, _, ratings = m8.get_dataset_2()
recomm_list = np.array(ratings)

prediction0 = recommender0(recomm_list)
prediction1 = recommender1(recomm_list)
# hybrid 추천
weight = [0.8, 0.2]
predictions = prediction0 * weight[0] + prediction1 * weight[1]
# 결과비교
recomm0_rmse = m8.RMSE2(ratings['rating'], prediction0)
recomm1_rmse = m8.RMSE2(ratings['rating'], prediction1)
hybrid_rmse = m8.RMSE2(ratings['rating'], predictions)

print(f"recomm0: {recomm0_rmse:.4f} / recomm1: {recomm1_rmse:.4f} / hybrid: {hybrid_rmse:.4f}")

recomm0: 1.6993 / recomm1: 1.6992 / hybrid: 1.5691


하이브리드 추천시스템(CF와 MF의 결합)

In [44]:
from sklearn.model_selection import train_test_split
import importlib
importlib.reload(m8) # 모듈 다시 불러오기 ( 변경사항 있으면 다시 불러와줘야 함 )

_, _, ratings = m8.get_dataset_2()
ratings_train, ratings_test, _, _ = train_test_split(ratings, ratings['rating'], train_size=0.2, random_state=1)
recomm_list = np.array(ratings_test)
rating_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

mf_kwargs = {
    'ratings': rating_matrix,
    'K': 200,
    'alpha': 0.001,
    'beta': 0.02,
    'iterations':100,
    'verbose':True
  }

mf = m8.NEW_MF(**mf_kwargs)
mf.set_test(ratings_test)
train_process = mf.train()

Iteration 10 / Train RMSE = 1.0250
Iteration 20 / Train RMSE = 0.9836
Iteration 30 / Train RMSE = 0.9605
Iteration 40 / Train RMSE = 0.9453
Iteration 50 / Train RMSE = 0.9342
Iteration 60 / Train RMSE = 0.9255
Iteration 70 / Train RMSE = 0.9183
Iteration 80 / Train RMSE = 0.9117
Iteration 90 / Train RMSE = 0.9054
Iteration 100 / Train RMSE = 0.8987


In [122]:
# CF
cf_kwargs = {
  'ratings': ratings,
  'use_split': False,
  'ratings_train': ratings_train,
  'ratings_test': ratings_test
}

cf = m8.CF(**cf_kwargs)

def recommender_CF(recomm_list, simil, neighbor_size):
  recommendations = [cf.CF_knn_bias_sig(user, movie, simil, neighbor_size) 
                     for (user, movie) in recomm_list]
  return np.array(recommendations)

# MF
def recommender_MF(recomm_list):
  recommendations = [mf.get_one_prediction(user, movie) for (user, movie) in recomm_list]
  return np.array(recommendations)

recomm_list = np.array(ratings_test.iloc[:, [0,1]])
prediction_CF = recommender_CF(recomm_list, simil='cosine', neighbor_size=30)
prediction_MF = recommender_MF(recomm_list)

weight = [0.8, 0.2]
prediction_hybrid = prediction_MF * weight[0] + prediction_CF * weight[1]

rmse_cf = m8.RMSE2(ratings_test.iloc[:, 2], prediction_CF)
rmse_mf = m8.RMSE2(ratings_test.iloc[:, 2], prediction_MF)
rmse_hybrid = m8.RMSE2(ratings_test.iloc[:, 2], prediction_hybrid)
print(f"rmse of , {weight[1]}cf : {rmse_cf} / {weight[0]}mf : {rmse_mf} / hybrid : {rmse_hybrid}") 


rmse of , 0.2cf : 1.0381060451579396 / 0.8mf : 0.9729583662137039 / hybrid : 0.9744075717727295


계속해서 남아있는 의문은, 기존에 없는 user id의 경우 예측이 불가능하다는 것.  